# 일단 데이터 맞나 체크


In [2]:
import os, glob

folder = "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725"

imgs = set([os.path.splitext(os.path.basename(f))[0] 
            for f in glob.glob(os.path.join(folder, "*.jpg"))])
lbls = set([os.path.splitext(os.path.basename(f))[0] 
            for f in glob.glob(os.path.join(folder, "*.txt"))])

only_imgs = imgs - lbls
only_lbls = lbls - imgs
matched   = imgs & lbls

print(f"=== {folder} ===")
print(f"총 이미지 수: {len(imgs)}")
print(f"총 라벨 수  : {len(lbls)}")
print(f"매칭된 쌍   : {len(matched)}")
print(f"라벨 없는 이미지: {len(only_imgs)}개")
for f in sorted(only_imgs):
    print("   -", f+".jpg")
print(f"이미지 없는 라벨: {len(only_lbls)}개")
for f in sorted(only_lbls):
    print("   -", f+".txt")


=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250725 ===
총 이미지 수: 2080
총 라벨 수  : 2080
매칭된 쌍   : 2080
라벨 없는 이미지: 0개
이미지 없는 라벨: 0개


In [4]:
import os, glob
from collections import Counter

folder = "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725"

label_files = glob.glob(os.path.join(folder, "*.txt"))
report = []

for lf in sorted(label_files):
    if os.path.basename(lf) == "classes.txt":  # 제외
        continue
    counts = Counter()
    with open(lf, "r") as f:
        for line in f:
            if line.strip():
                cid = line.split()[0]
                if cid.isdigit():
                    counts[int(cid)] += 1
    report.append((os.path.basename(lf), sum(counts.values()), dict(counts)))

# 요약 출력
print(f"총 라벨 파일: {len(report)}")
for fname, total, counts in report[:20]:  # 처음 20개만 샘플 출력
    print(f"{fname}: 객체 {total}개 {counts}")

# 전체 클래스 분포 합계
global_counts = Counter()
for _, _, c in report:
    global_counts.update(c)

print("\n=== 전체 클래스 분포 ===")
for cid, c in sorted(global_counts.items()):
    print(f"class {cid}: {c}개")


총 라벨 파일: 2080
20250725_170115_0000_000001_f000000_1920x1536.txt: 객체 1개 {0: 1}
20250725_170115_0000_000002_f000030_1920x1536.txt: 객체 1개 {0: 1}
20250725_170115_0000_000003_f000060_1920x1536.txt: 객체 1개 {0: 1}
20250725_170115_0000_000004_f000090_1920x1536.txt: 객체 1개 {0: 1}
20250725_170115_0000_000005_f000120_1920x1536.txt: 객체 1개 {0: 1}
20250725_170115_0000_000006_f000150_1920x1536.txt: 객체 1개 {0: 1}
20250725_170115_0000_000007_f000180_1920x1536.txt: 객체 1개 {0: 1}
20250725_170115_0000_000008_f000210_1920x1536.txt: 객체 1개 {0: 1}
20250725_170115_0000_000009_f000240_1920x1536.txt: 객체 0개 {}
20250725_170115_0000_000010_f000270_1920x1536.txt: 객체 0개 {}
20250725_170115_0000_000011_f000300_1920x1536.txt: 객체 0개 {}
20250725_170115_0000_000012_f000330_1920x1536.txt: 객체 0개 {}
20250725_170115_0000_000013_f000360_1920x1536.txt: 객체 1개 {0: 1}
20250725_170115_0000_000014_f000390_1920x1536.txt: 객체 1개 {0: 1}
20250725_170115_0000_000015_f000420_1920x1536.txt: 객체 1개 {0: 1}
20250725_170115_0000_000016_f000450_1920x1

In [5]:
import os, glob

folder = "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250721"

# 매핑 규칙 정의
mapping = {0: 2, 1: 0, 2: 1}

label_files = glob.glob(os.path.join(folder, "*.txt"))

for lf in label_files:
    if os.path.basename(lf) == "classes.txt":
        continue
    lines_out = []
    with open(lf, "r") as f:
        for line in f:
            parts = line.strip().split()
            if not parts:
                continue
            cid = int(parts[0])
            if cid in mapping:
                parts[0] = str(mapping[cid])
            else:
                # 혹시 mapping에 없는 클래스는 그대로 둠
                parts[0] = str(cid)
            lines_out.append(" ".join(parts))
    with open(lf, "w") as f:
        f.write("\n".join(lines_out) + "\n")

print(f"[완료] {len(label_files)}개 라벨 파일 클래스 ID 변환 완료")


[완료] 5834개 라벨 파일 클래스 ID 변환 완료


In [7]:
import os, glob, shutil

src = "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725"
dst = "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725_txt_backup"
os.makedirs(dst, exist_ok=True)

for f in glob.glob(os.path.join(src, "*.txt")):
    shutil.copy2(f, os.path.join(dst, os.path.basename(f)))

print(f"[완료] txt 파일 {len(glob.glob(os.path.join(dst,'*.txt')))}개 백업됨 → {dst}")


[완료] txt 파일 2080개 백업됨 → /home/dw/ws_job_msislab/Golf_Project/data/Data_20250725_txt_backup


In [8]:
import os, glob

folder = "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725"

keep_classes = {0, 1, 2}  # 유지할 클래스

label_files = glob.glob(os.path.join(folder, "*.txt"))

modified, emptied = 0, 0
for lf in label_files:
    if os.path.basename(lf) == "classes.txt":
        continue
    new_lines = []
    with open(lf, "r") as f:
        for line in f:
            parts = line.strip().split()
            if not parts:
                continue
            cid = int(parts[0])
            if cid in keep_classes:
                new_lines.append(line.strip())
    # 라벨 파일 업데이트
    with open(lf, "w") as f:
        if new_lines:
            f.write("\n".join(new_lines) + "\n")
            modified += 1
        else:
            f.write("")  # 아예 비우기
            emptied += 1

print(f"[완료] {len(label_files)}개 라벨 파일 처리")
print(f" - 수정된 파일: {modified}")
print(f" - 완전히 비워진 파일(모든 객체가 삭제됨): {emptied}")


[완료] 2080개 라벨 파일 처리
 - 수정된 파일: 1348
 - 완전히 비워진 파일(모든 객체가 삭제됨): 732


In [10]:
import os, glob

def count_empty_labels(folder):
    label_files = glob.glob(os.path.join(folder, "*.txt"))
    empty_files = []
    for lf in label_files:
        if os.path.basename(lf) == "classes.txt":
            continue
        with open(lf, "r") as f:
            lines = [ln.strip() for ln in f if ln.strip()]
        if len(lines) == 0:
            empty_files.append(os.path.basename(lf))
    return empty_files

folders = [
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250721",
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725",
]

for folder in folders:
    empties = count_empty_labels(folder)
    print(f"=== {folder} ===")
    print(f"빈 라벨 파일 개수: {len(empties)}")
    if len(empties) > 0:
        print("예시:", empties[:20])  # 많으면 처음 20개만 표시


=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721 ===
빈 라벨 파일 개수: 3040
예시: ['20250721_053640_0032_000063_f000620_1920x1536_3889.txt', '20250721_053640_0029_000013_f000120_1920x1536_2308.txt', '20250721_053640_0035_000031_f000300_1920x1536_4762.txt', '20250721_053640_0034_000156_f001550_1920x1536_4613.txt', '20250721_053640_0023_000044_f000430_1920x1536_2031.txt', '20250721_053640_0032_000162_f001610_1920x1536_4169.txt', '20250721_053640_0007_000151_f001500_1920x1536_871.txt', '20250721_053640_0034_000128_f001270_1920x1536_4542.txt', '20250721_053640_0030_000030_f000290_1920x1536_2825.txt', '20250721_053640_0032_000167_f001660_1920x1536_4183.txt', '20250721_053640_0023_000096_f000950_1920x1536_2154.txt', '20250721_053640_0023_000016_f000150_1920x1536_1950.txt', '20250721_053640_0035_000101_f001000_1920x1536_4937.txt', '20250721_053640_0030_000136_f001350_1920x1536_3123.txt', '20250721_053640_0036_000155_f001540_1920x1536_5554.txt', '20250721_053640_0034_000161_f001600_1920x153

In [11]:
import os, glob

def check_empty_labels(folder):
    label_files = glob.glob(os.path.join(folder, "*.txt"))
    total = 0
    empty = 0
    empty_list = []
    for lf in label_files:
        if os.path.basename(lf) == "classes.txt":
            continue
        total += 1
        with open(lf, "r") as f:
            lines = [ln.strip() for ln in f if ln.strip()]
        if len(lines) == 0:
            empty += 1
            empty_list.append(os.path.basename(lf))
    ratio = (empty / total * 100) if total > 0 else 0
    return total, empty, ratio, empty_list

folders = [
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250721",
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725",
]

for folder in folders:
    total, empty, ratio, empty_list = check_empty_labels(folder)
    print(f"=== {folder} ===")
    print(f"총 라벨 파일 수: {total}")
    print(f"빈 라벨 파일 수: {empty} ({ratio:.2f}%)")
    if empty > 0:
        print("예시:", empty_list[:20])  # 일부 샘플만 표시


=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721 ===
총 라벨 파일 수: 5834
빈 라벨 파일 수: 3040 (52.11%)
예시: ['20250721_053640_0032_000063_f000620_1920x1536_3889.txt', '20250721_053640_0029_000013_f000120_1920x1536_2308.txt', '20250721_053640_0035_000031_f000300_1920x1536_4762.txt', '20250721_053640_0034_000156_f001550_1920x1536_4613.txt', '20250721_053640_0023_000044_f000430_1920x1536_2031.txt', '20250721_053640_0032_000162_f001610_1920x1536_4169.txt', '20250721_053640_0007_000151_f001500_1920x1536_871.txt', '20250721_053640_0034_000128_f001270_1920x1536_4542.txt', '20250721_053640_0030_000030_f000290_1920x1536_2825.txt', '20250721_053640_0032_000167_f001660_1920x1536_4183.txt', '20250721_053640_0023_000096_f000950_1920x1536_2154.txt', '20250721_053640_0023_000016_f000150_1920x1536_1950.txt', '20250721_053640_0035_000101_f001000_1920x1536_4937.txt', '20250721_053640_0030_000136_f001350_1920x1536_3123.txt', '20250721_053640_0036_000155_f001540_1920x1536_5554.txt', '20250721_053640_0034

In [12]:
import os, glob, random

def reduce_empty_labels(folder, target_ratio=0.2, seed=42):
    random.seed(seed)
    label_files = [f for f in glob.glob(os.path.join(folder, "*.txt")) 
                   if os.path.basename(f) != "classes.txt"]

    empty_files = []
    non_empty_files = []
    for lf in label_files:
        with open(lf, "r") as f:
            lines = [ln.strip() for ln in f if ln.strip()]
        if len(lines) == 0:
            empty_files.append(lf)
        else:
            non_empty_files.append(lf)

    total = len(label_files)
    max_empty_allowed = int(total * target_ratio)

    print(f"\n=== {folder} ===")
    print(f"전체 라벨 파일: {total}")
    print(f"현재 빈 라벨: {len(empty_files)}")
    print(f"목표 비율 {target_ratio*100:.0f}% → 최대 {max_empty_allowed}개 허용")

    if len(empty_files) > max_empty_allowed:
        # 샘플링해서 일부만 유지
        keep = set(random.sample(empty_files, max_empty_allowed))
        remove = [f for f in empty_files if f not in keep]

        for f in remove:
            os.remove(f)
        print(f"{len(remove)}개 빈 라벨 파일 삭제 완료, {len(keep)}개 유지됨")
    else:
        print("이미 목표 이하라서 아무것도 삭제 안 함.")

# 실행
reduce_empty_labels("/home/dw/ws_job_msislab/Golf_Project/data/Data_20250721", target_ratio=0.2)
reduce_empty_labels("/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725", target_ratio=0.2)



=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721 ===
전체 라벨 파일: 5834
현재 빈 라벨: 3040
목표 비율 20% → 최대 1166개 허용
1874개 빈 라벨 파일 삭제 완료, 1166개 유지됨

=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250725 ===
전체 라벨 파일: 2080
현재 빈 라벨: 732
목표 비율 20% → 최대 416개 허용
316개 빈 라벨 파일 삭제 완료, 416개 유지됨


In [13]:
import os, glob

def check_empty_labels(folder):
    label_files = glob.glob(os.path.join(folder, "*.txt"))
    total = 0
    empty = 0
    empty_list = []
    for lf in label_files:
        if os.path.basename(lf) == "classes.txt":
            continue
        total += 1
        with open(lf, "r") as f:
            lines = [ln.strip() for ln in f if ln.strip()]
        if len(lines) == 0:
            empty += 1
            empty_list.append(os.path.basename(lf))
    ratio = (empty / total * 100) if total > 0 else 0
    return total, empty, ratio, empty_list

folders = [
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250721",
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725",
]

for folder in folders:
    total, empty, ratio, empty_list = check_empty_labels(folder)
    print(f"=== {folder} ===")
    print(f"총 라벨 파일 수: {total}")
    print(f"빈 라벨 파일 수: {empty} ({ratio:.2f}%)")
    if empty > 0:
        print("예시:", empty_list[:20])  # 일부 샘플만 표시


=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721 ===
총 라벨 파일 수: 3960
빈 라벨 파일 수: 1166 (29.44%)
예시: ['20250721_053640_0029_000013_f000120_1920x1536_2308.txt', '20250721_053640_0035_000031_f000300_1920x1536_4762.txt', '20250721_053640_0023_000044_f000430_1920x1536_2031.txt', '20250721_053640_0007_000151_f001500_1920x1536_871.txt', '20250721_053640_0030_000030_f000290_1920x1536_2825.txt', '20250721_053640_0023_000096_f000950_1920x1536_2154.txt', '20250721_053640_0030_000136_f001350_1920x1536_3123.txt', '20250721_053640_0036_000155_f001540_1920x1536_5554.txt', '20250721_053640_0032_000119_f001180_1920x1536_4052.txt', '20250721_053640_0007_000161_f001600_1920x1536_32.txt', '20250721_053640_0036_000170_f001690_1920x1536_5595.txt', '20250721_053640_0032_000108_f001070_1920x1536_4019.txt', '20250721_053640_0034_000117_f001160_1920x1536_4510.txt', '20250721_053640_0034_000065_f000640_1920x1536_4382.txt', '20250721_053640_0032_000045_f000440_1920x1536_3835.txt', '20250721_053640_0032_0

In [15]:
import os, glob, random

def enforce_empty_ratio(folder, target_ratio=0.2, seed=42):
    random.seed(seed)
    # classes.txt 제외
    label_files = [f for f in glob.glob(os.path.join(folder, "*.txt"))
                   if os.path.basename(f) != "classes.txt"]

    # 현재 빈/비빈 분리
    empty = []
    non_empty = []
    for lf in label_files:
        with open(lf, "r") as f:
            lines = [ln.strip() for ln in f if ln.strip()]
        (empty if len(lines)==0 else non_empty).append(lf)

    total = len(label_files)
    target_empty = int(total * target_ratio)  # 정확히 이만큼 남긴다
    cur_empty = len(empty)

    print(f"\n=== {folder} ===")
    print(f"현재 총 라벨: {total}")
    print(f"현재 빈 라벨: {cur_empty} ({cur_empty/total*100:.2f}%)")
    print(f"목표 빈 라벨: {target_empty} ({target_ratio*100:.0f}%)")

    if cur_empty == target_empty:
        print("이미 목표와 동일. 아무 것도 안 함.")
        return

    if cur_empty > target_empty:
        # 너무 많으면 (cur_empty - target_empty)개를 지운다
        to_remove = cur_empty - target_empty
        remove_set = set(random.sample(empty, to_remove))
        removed = 0
        for lf in remove_set:
            stem = os.path.splitext(lf)[0]
            # 라벨 삭제
            if os.path.exists(lf):
                os.remove(lf)
            # 같은 이름의 이미지도 삭제
            for ext in [".jpg",".jpeg",".png",".bmp",".webp"]:
                img = stem + ext
                if os.path.exists(img):
                    os.remove(img)
            removed += 1
        print(f"추가 삭제: {removed}개 (빈 라벨 + 짝 이미지)")

    else:
        # 빈 라벨이 목표보다 적으면? (보통 필요 없음)
        # 비어있는 라벨을 늘릴 수는 없으니 안내만.
        print("빈 라벨이 목표보다 적습니다. 추가 작업 없음.")

    # 최종 재계산
    label_files2 = [f for f in glob.glob(os.path.join(folder, "*.txt"))
                    if os.path.basename(f) != "classes.txt"]
    empty2 = 0
    for lf in label_files2:
        with open(lf, "r") as f:
            if any(ln.strip() for ln in f):
                continue
        empty2 += 1
    total2 = len(label_files2)
    print(f"최종 빈 라벨: {empty2} / {total2} ({(empty2/total2*100 if total2 else 0):.2f}%)")

# 실행
enforce_empty_ratio("/home/dw/ws_job_msislab/Golf_Project/data/Data_20250721", target_ratio=0.2)
enforce_empty_ratio("/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725", target_ratio=0.2)




=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721 ===
현재 총 라벨: 3960
현재 빈 라벨: 1166 (29.44%)
목표 빈 라벨: 792 (20%)
추가 삭제: 374개 (빈 라벨 + 짝 이미지)
최종 빈 라벨: 792 / 3586 (22.09%)

=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250725 ===
현재 총 라벨: 1764
현재 빈 라벨: 416 (23.58%)
목표 빈 라벨: 352 (20%)
추가 삭제: 64개 (빈 라벨 + 짝 이미지)
최종 빈 라벨: 352 / 1700 (20.71%)


In [16]:
import os, glob

def check_empty_labels(folder):
    label_files = glob.glob(os.path.join(folder, "*.txt"))
    total = 0
    empty = 0
    empty_list = []
    for lf in label_files:
        if os.path.basename(lf) == "classes.txt":
            continue
        total += 1
        with open(lf, "r") as f:
            lines = [ln.strip() for ln in f if ln.strip()]
        if len(lines) == 0:
            empty += 1
            empty_list.append(os.path.basename(lf))
    ratio = (empty / total * 100) if total > 0 else 0
    return total, empty, ratio, empty_list

folders = [
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250721",
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725",
]

for folder in folders:
    total, empty, ratio, empty_list = check_empty_labels(folder)
    print(f"=== {folder} ===")
    print(f"총 라벨 파일 수: {total}")
    print(f"빈 라벨 파일 수: {empty} ({ratio:.2f}%)")
    if empty > 0:
        print("예시:", empty_list[:20])  # 일부 샘플만 표시


=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721 ===
총 라벨 파일 수: 3586
빈 라벨 파일 수: 792 (22.09%)
예시: ['20250721_053640_0029_000013_f000120_1920x1536_2308.txt', '20250721_053640_0007_000151_f001500_1920x1536_871.txt', '20250721_053640_0030_000030_f000290_1920x1536_2825.txt', '20250721_053640_0023_000096_f000950_1920x1536_2154.txt', '20250721_053640_0032_000119_f001180_1920x1536_4052.txt', '20250721_053640_0036_000170_f001690_1920x1536_5595.txt', '20250721_053640_0032_000108_f001070_1920x1536_4019.txt', '20250721_053640_0034_000117_f001160_1920x1536_4510.txt', '20250721_053640_0032_000045_f000440_1920x1536_3835.txt', '20250721_053640_0032_000033_f000320_1920x1536_3802.txt', '20250721_053640_0032_000006_f000050_1920x1536_3725.txt', '20250721_053640_0030_000137_f001360_1920x1536_3126.txt', '20250721_053640_0035_000006_f000050_1920x1536_4693.txt', '20250721_053640_0035_000021_f000200_1920x1536_4732.txt', '20250721_053640_0023_000004_f000030_1920x1536_1912.txt', '20250721_053640_0022_

In [17]:
import os, glob, random

def trim_empty_to_ratio(folder, target_ratio=0.20, seed=42):
    random.seed(seed)
    labels = [f for f in glob.glob(os.path.join(folder, "*.txt"))
              if os.path.basename(f) != "classes.txt"]

    empty = []
    for lf in labels:
        with open(lf, "r") as f:
            if not any(ln.strip() for ln in f):
                empty.append(lf)

    total = len(labels)
    # 가장 근접한 정수 개수로 맞춤(반올림)
    target_empty = int(total * target_ratio + 0.5)
    cur_empty = len(empty)

    print(f"\n=== {folder} ===")
    print(f"현재 총 라벨: {total}")
    print(f"현재 빈 라벨: {cur_empty} ({cur_empty/total*100:.2f}%)")
    print(f"목표 빈 라벨: {target_empty} ({target_ratio*100:.0f}%)")

    if cur_empty > target_empty:
        # 초과분 만큼만 랜덤 삭제
        to_remove = cur_empty - target_empty
        rm = set(random.sample(empty, to_remove))
        for lf in rm:
            stem = os.path.splitext(lf)[0]
            if os.path.exists(lf): os.remove(lf)
            for ext in [".jpg",".jpeg",".png",".bmp",".webp"]:
                ip = stem + ext
                if os.path.exists(ip): os.remove(ip)
        print(f"추가 삭제: {to_remove}개 (빈 라벨 + 짝 이미지)")
    else:
        print("추가 삭제 불필요.")

    # 최종 확인
    labels2 = [f for f in glob.glob(os.path.join(folder, "*.txt"))
               if os.path.basename(f) != "classes.txt"]
    empty2 = 0
    for lf in labels2:
        with open(lf, "r") as f:
            if not any(ln.strip() for ln in f): empty2 += 1
    total2 = len(labels2)
    print(f"최종 빈 라벨: {empty2} / {total2} ({(empty2/total2*100 if total2 else 0):.2f}%)")

# 실행: 21, 25 둘 다 정확히 20%로 보정
trim_empty_to_ratio("/home/dw/ws_job_msislab/Golf_Project/data/Data_20250721", target_ratio=0.20)
trim_empty_to_ratio("/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725", target_ratio=0.20)



=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721 ===
현재 총 라벨: 3586
현재 빈 라벨: 792 (22.09%)
목표 빈 라벨: 717 (20%)
추가 삭제: 75개 (빈 라벨 + 짝 이미지)
최종 빈 라벨: 717 / 3511 (20.42%)

=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250725 ===
현재 총 라벨: 1700
현재 빈 라벨: 352 (20.71%)
목표 빈 라벨: 340 (20%)
추가 삭제: 12개 (빈 라벨 + 짝 이미지)
최종 빈 라벨: 340 / 1688 (20.14%)


In [18]:
import os, glob

def check_empty_labels(folder):
    label_files = glob.glob(os.path.join(folder, "*.txt"))
    total = 0
    empty = 0
    empty_list = []
    for lf in label_files:
        if os.path.basename(lf) == "classes.txt":
            continue
        total += 1
        with open(lf, "r") as f:
            lines = [ln.strip() for ln in f if ln.strip()]
        if len(lines) == 0:
            empty += 1
            empty_list.append(os.path.basename(lf))
    ratio = (empty / total * 100) if total > 0 else 0
    return total, empty, ratio, empty_list

folders = [
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250721",
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725",
]

for folder in folders:
    total, empty, ratio, empty_list = check_empty_labels(folder)
    print(f"=== {folder} ===")
    print(f"총 라벨 파일 수: {total}")
    print(f"빈 라벨 파일 수: {empty} ({ratio:.2f}%)")
    if empty > 0:
        print("예시:", empty_list[:20])  # 일부 샘플만 표시


=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721 ===
총 라벨 파일 수: 3511
빈 라벨 파일 수: 717 (20.42%)
예시: ['20250721_053640_0029_000013_f000120_1920x1536_2308.txt', '20250721_053640_0007_000151_f001500_1920x1536_871.txt', '20250721_053640_0030_000030_f000290_1920x1536_2825.txt', '20250721_053640_0023_000096_f000950_1920x1536_2154.txt', '20250721_053640_0032_000119_f001180_1920x1536_4052.txt', '20250721_053640_0036_000170_f001690_1920x1536_5595.txt', '20250721_053640_0034_000117_f001160_1920x1536_4510.txt', '20250721_053640_0032_000045_f000440_1920x1536_3835.txt', '20250721_053640_0032_000033_f000320_1920x1536_3802.txt', '20250721_053640_0032_000006_f000050_1920x1536_3725.txt', '20250721_053640_0030_000137_f001360_1920x1536_3126.txt', '20250721_053640_0035_000006_f000050_1920x1536_4693.txt', '20250721_053640_0035_000021_f000200_1920x1536_4732.txt', '20250721_053640_0023_000004_f000030_1920x1536_1912.txt', '20250721_053640_0022_000153_f001520_1920x1536_1824.txt', '20250721_053640_0022_

In [19]:
import os, glob, re

FOLDERS = [
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250721",
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725",
]

IMG_EXTS = (".jpg",".jpeg",".png",".bmp",".webp")

def has_image(stem):
    for e in IMG_EXTS:
        if os.path.exists(stem + e):
            return True
    return False

def check_line(raw):
    """
    한 줄 검사 → (ok:bool, reasons:list[str])
    YOLO 형식 기대: "<int_class> <float> <float> <float> <float>"
    """
    reasons = []
    s = raw.strip()

    if not s:
        # 빈 줄은 그냥 무시(라벨 없는 파일 판단은 별도로)
        return True, reasons

    # 흔한 오류: 콤마/탭/여러 공백
    if "," in s:
        reasons.append("comma_found")
    if "\t" in s:
        reasons.append("tab_found")
    if re.search(r"[A-Za-z]", s):
        reasons.append("alpha_token_found")  # ex) 'divot'

    parts = re.split(r"\s+", s)
    if len(parts) != 5:
        reasons.append("token_count_not_5")
        return False, reasons

    # class id
    try:
        cid = int(parts[0])
    except:
        reasons.append("class_not_int")
        return False, reasons

    # coords
    nums = []
    for p in parts[1:]:
        try:
            nums.append(float(p))
        except:
            reasons.append("coord_not_float")
            return False, reasons

    x,y,w,h = nums

    # 값 검증
    if any(v < 0 for v in (x,y,w,h)):
        reasons.append("negative_value")

    if any(v > 1 for v in (x,y,w,h)):
        reasons.append("value_gt_1")  # 정규화 안 됐을 가능성

    # 폭/높이 0 또는 이상치
    if w <= 0 or h <= 0:
        reasons.append("non_positive_wh")

    # 정상/경고 판단
    ok = len([r for r in reasons if r in
              ("token_count_not_5","class_not_int","coord_not_float")]) == 0
    return ok, reasons

def check_folder(folder):
    stats = {
        "total_label_files": 0,
        "empty_label_files": 0,
        "no_image_match": 0,
        "bad_lines_files": 0,
        "bad_lines_count": 0,
        "reasons_count": {},
    }
    samples = []  # (file, line_no, reason, line)

    label_files = sorted([p for p in glob.glob(os.path.join(folder, "*.txt"))
                          if os.path.basename(p) != "classes.txt"])
    stats["total_label_files"] = len(label_files)

    for lf in label_files:
        stem = os.path.splitext(lf)[0]
        if not has_image(stem):
            stats["no_image_match"] += 1

        with open(lf, "r", encoding="utf-8", errors="ignore") as f:
            lines = f.readlines()

        # 빈 라벨 파일
        nonempty = [ln for ln in lines if ln.strip()]
        if len(nonempty) == 0:
            stats["empty_label_files"] += 1
            continue

        bad_in_this_file = False
        for i, raw in enumerate(lines, start=1):
            s = raw.strip()
            if not s:
                continue
            ok, reasons = check_line(raw)
            if not ok or any(r in reasons for r in
                             ("comma_found","tab_found","alpha_token_found",
                              "negative_value","value_gt_1","non_positive_wh")):
                bad_in_this_file = True
                stats["bad_lines_count"] += 1
                # 이유 카운트
                for r in reasons:
                    stats["reasons_count"][r] = stats["reasons_count"].get(r, 0) + 1
                if len(samples) < 30:
                    samples.append((lf, i, ",".join(reasons), raw.rstrip()))

        if bad_in_this_file:
            stats["bad_lines_files"] += 1

    # 출력
    print(f"\n=== {folder} ===")
    print(f"총 라벨 파일: {stats['total_label_files']}")
    print(f"빈 라벨 파일: {stats['empty_label_files']}")
    print(f"매칭 이미지 없는 라벨: {stats['no_image_match']}")
    print(f"문제 라인이 있는 라벨 파일 수: {stats['bad_lines_files']}")
    print(f"문제 라인 총 개수: {stats['bad_lines_count']}")
    if stats["reasons_count"]:
        print("문제 유형 카운트:")
        for k,v in sorted(stats["reasons_count"].items(), key=lambda x:-x[1]):
            print(f"  - {k}: {v}")
    if samples:
        print("\n문제 샘플(최대 30개):")
        for fpath,ln,why,txt in samples:
            print(f"  {fpath}:{ln} [{why}] -> {txt}")

for d in FOLDERS:
    check_folder(d)



=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721 ===
총 라벨 파일: 3511
빈 라벨 파일: 717
매칭 이미지 없는 라벨: 0
문제 라인이 있는 라벨 파일 수: 0
문제 라인 총 개수: 0

=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250725 ===
총 라벨 파일: 1688
빈 라벨 파일: 340
매칭 이미지 없는 라벨: 0
문제 라인이 있는 라벨 파일 수: 0
문제 라인 총 개수: 0


In [20]:
import os, glob

FOLDERS = [
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250721",
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725",
]

IMG_EXTS = (".jpg",".jpeg",".png",".bmp",".webp")

def check_pairs(folder):
    imgs = {}
    lbls = {}

    for e in IMG_EXTS:
        for p in glob.glob(os.path.join(folder, "*" + e)):
            stem = os.path.splitext(os.path.basename(p))[0]
            imgs[stem] = p

    for p in glob.glob(os.path.join(folder, "*.txt")):
        if os.path.basename(p) == "classes.txt":
            continue
        stem = os.path.splitext(os.path.basename(p))[0]
        lbls[stem] = p

    only_imgs = sorted(set(imgs.keys()) - set(lbls.keys()))
    only_lbls = sorted(set(lbls.keys()) - set(imgs.keys()))

    print(f"\n=== {folder} ===")
    print(f"총 이미지: {len(imgs)}, 총 라벨: {len(lbls)}")
    print(f"이미지만 있고 라벨 없는 파일: {len(only_imgs)}개")
    if only_imgs:
        print("  예시:", only_imgs[:20])
    print(f"라벨만 있고 이미지 없는 파일: {len(only_lbls)}개")
    if only_lbls:
        print("  예시:", only_lbls[:20])

for d in FOLDERS:
    check_pairs(d)



=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721 ===
총 이미지: 5385, 총 라벨: 3511
이미지만 있고 라벨 없는 파일: 1874개
  예시: ['20250721_053640_0006_000006_f000050_1920x1536_13', '20250721_053640_0006_000006_f000050_1920x1536_14', '20250721_053640_0006_000009_f000080_1920x1536_22', '20250721_053640_0006_000009_f000080_1920x1536_24', '20250721_053640_0006_000013_f000120_1920x1536_34', '20250721_053640_0006_000013_f000120_1920x1536_36', '20250721_053640_0006_000015_f000140_1920x1536_40', '20250721_053640_0006_000015_f000140_1920x1536_41', '20250721_053640_0006_000015_f000140_1920x1536_42', '20250721_053640_0006_000025_f000240_1920x1536_68', '20250721_053640_0006_000027_f000260_1920x1536_74', '20250721_053640_0006_000035_f000340_1920x1536_4', '20250721_053640_0006_000047_f000460_1920x1536_124', '20250721_053640_0006_000047_f000460_1920x1536_125', '20250721_053640_0006_000047_f000460_1920x1536_126', '20250721_053640_0006_000048_f000470_1920x1536_127', '20250721_053640_0006_000048_f000470_1920x153

In [21]:
import os, glob

FOLDERS = [
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250721",
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725",
]

IMG_EXTS = (".jpg",".jpeg",".png",".bmp",".webp")

def delete_unmatched_images(folder):
    # 라벨 stem 집합 생성 (*.txt, classes.txt 제외)
    label_stems = set()
    for p in glob.glob(os.path.join(folder, "*.txt")):
        if os.path.basename(p) == "classes.txt":
            continue
        label_stems.add(os.path.splitext(os.path.basename(p))[0])

    # 이미지 중 라벨이 없는 것 삭제
    imgs_total = 0
    deleted = 0
    samples = []
    for ext in IMG_EXTS:
        for ip in glob.glob(os.path.join(folder, f"*{ext}")):
            imgs_total += 1
            stem = os.path.splitext(os.path.basename(ip))[0]
            if stem not in label_stems:
                os.remove(ip)
                deleted += 1
                if len(samples) < 20:
                    samples.append(os.path.basename(ip))

    print(f"\n=== {folder} ===")
    print(f"총 이미지: {imgs_total}")
    print(f"라벨 없는 이미지 삭제: {deleted}개")
    if samples:
        print("삭제 예시:", samples)

for d in FOLDERS:
    delete_unmatched_images(d)



=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721 ===
총 이미지: 5385
라벨 없는 이미지 삭제: 1874개
삭제 예시: ['20250721_053640_0036_000085_f000840_1920x1536_5374.jpg', '20250721_053640_0021_000046_f000450_1920x1536_1072.jpg', '20250721_053640_0021_000175_f001740_1920x1536_1403.jpg', '20250721_053640_0032_000118_f001170_1920x1536_4048.jpg', '20250721_053640_0021_000172_f001710_1920x1536_1396.jpg', '20250721_053640_0031_000055_f000540_1920x1536_132.jpg', '20250721_053640_0032_000155_f001540_1920x1536_4147.jpg', '20250721_053640_0036_000061_f000600_1920x1536_5325.jpg', '20250721_053640_0032_000145_f001440_1920x1536_4125.jpg', '20250721_053640_0036_000136_f001350_1920x1536_5501.jpg', '20250721_053640_0032_000050_f000490_1920x1536_3852.jpg', '20250721_053640_0021_000119_f001180_1920x1536_1256.jpg', '20250721_053640_0036_000130_f001290_1920x1536_5489.jpg', '20250721_053640_0032_000078_f000770_1920x1536_3935.jpg', '20250721_053640_0036_000018_f000170_1920x1536_5204.jpg', '20250721_053640_0034_0001

In [22]:
import os, glob

FOLDERS = [
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250721",
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725",
]

IMG_EXTS = (".jpg",".jpeg",".png",".bmp",".webp")

def check_pairs(folder):
    imgs = {}
    lbls = {}

    for e in IMG_EXTS:
        for p in glob.glob(os.path.join(folder, "*" + e)):
            stem = os.path.splitext(os.path.basename(p))[0]
            imgs[stem] = p

    for p in glob.glob(os.path.join(folder, "*.txt")):
        if os.path.basename(p) == "classes.txt":
            continue
        stem = os.path.splitext(os.path.basename(p))[0]
        lbls[stem] = p

    only_imgs = sorted(set(imgs.keys()) - set(lbls.keys()))
    only_lbls = sorted(set(lbls.keys()) - set(imgs.keys()))

    print(f"\n=== {folder} ===")
    print(f"총 이미지: {len(imgs)}")
    print(f"총 라벨 : {len(lbls)}")
    print(f"이미지만 있고 라벨 없는 파일: {len(only_imgs)}개")
    print(f"라벨만 있고 이미지 없는 파일: {len(only_lbls)}개")

    if only_imgs:
        print("  예시:", only_imgs[:20])
    if only_lbls:
        print("  예시:", only_lbls[:20])

for d in FOLDERS:
    check_pairs(d)



=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721 ===
총 이미지: 3511
총 라벨 : 3511
이미지만 있고 라벨 없는 파일: 0개
라벨만 있고 이미지 없는 파일: 0개

=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250725 ===
총 이미지: 1688
총 라벨 : 1688
이미지만 있고 라벨 없는 파일: 0개
라벨만 있고 이미지 없는 파일: 0개


In [23]:
import os, glob

FOLDERS = [
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250721",
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725",
]

def check_empty_labels(folder):
    label_files = [p for p in glob.glob(os.path.join(folder, "*.txt"))
                   if os.path.basename(p) != "classes.txt"]

    total = len(label_files)
    empty = []
    for lf in label_files:
        with open(lf, "r") as f:
            lines = [ln.strip() for ln in f if ln.strip()]
        if len(lines) == 0:
            empty.append(os.path.basename(lf))

    ratio = (len(empty) / total * 100) if total > 0 else 0
    print(f"\n=== {folder} ===")
    print(f"총 라벨 파일 수: {total}")
    print(f"빈 라벨 파일 수: {len(empty)} ({ratio:.2f}%)")
    if empty:
        print("예시:", empty[:20])  # 샘플 일부만 출력

for d in FOLDERS:
    check_empty_labels(d)



=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721 ===
총 라벨 파일 수: 3511
빈 라벨 파일 수: 717 (20.42%)
예시: ['20250721_053640_0029_000013_f000120_1920x1536_2308.txt', '20250721_053640_0007_000151_f001500_1920x1536_871.txt', '20250721_053640_0030_000030_f000290_1920x1536_2825.txt', '20250721_053640_0023_000096_f000950_1920x1536_2154.txt', '20250721_053640_0032_000119_f001180_1920x1536_4052.txt', '20250721_053640_0036_000170_f001690_1920x1536_5595.txt', '20250721_053640_0034_000117_f001160_1920x1536_4510.txt', '20250721_053640_0032_000045_f000440_1920x1536_3835.txt', '20250721_053640_0032_000033_f000320_1920x1536_3802.txt', '20250721_053640_0032_000006_f000050_1920x1536_3725.txt', '20250721_053640_0030_000137_f001360_1920x1536_3126.txt', '20250721_053640_0035_000006_f000050_1920x1536_4693.txt', '20250721_053640_0035_000021_f000200_1920x1536_4732.txt', '20250721_053640_0023_000004_f000030_1920x1536_1912.txt', '20250721_053640_0022_000153_f001520_1920x1536_1824.txt', '20250721_053640_0022

In [24]:
import os, glob, re, sys
from collections import Counter, defaultdict

FOLDERS = [
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250721",
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725",
]
IMG_EXTS = (".jpg",".jpeg",".png",".bmp",".webp")

def has_image(stem):
    for e in IMG_EXTS:
        if os.path.exists(stem + e):
            return True
    return False

def check_line(raw):
    """ YOLO 기대: '<int> <float> <float> <float> <float>' """
    reasons = []
    s = raw.rstrip("\r\n")
    if not s.strip():
        return True, []  # 빈 줄은 무시(파일 empty 여부는 별도로 판단)

    if "\t" in s: reasons.append("has_tab")
    if "," in s:  reasons.append("has_comma")
    if re.search(r"[A-Za-z]", s): reasons.append("has_alpha")

    parts = s.split(' ')  # labelImg가 ' ' 기준 split 사용
    parts = [p for p in parts if p != ""]  # 연속 공백 보정
    if len(parts) != 5:
        reasons.append("token_count_not_5")
        return False, reasons

    # class
    try:
        int(parts[0])
    except:
        reasons.append("class_not_int")
        return False, reasons

    # coords
    nums = []
    for p in parts[1:]:
        try:
            nums.append(float(p))
        except:
            reasons.append("coord_not_float")
            return False, reasons
    x,y,w,h = nums

    if any(v < 0 for v in (x,y,w,h)):
        reasons.append("negative_value")
    if w <= 0 or h <= 0:
        reasons.append("non_positive_wh")
    if any(v > 1 for v in (x,y,w,h)):
        reasons.append("value_gt_1")  # 정규화 안됐을 가능성
    ok = not any(r in {"token_count_not_5","class_not_int","coord_not_float"} for r in reasons)
    return ok, reasons

def check_folder(folder, max_samples=50):
    label_files = sorted([p for p in glob.glob(os.path.join(folder, "*.txt"))
                          if os.path.basename(p) != "classes.txt"])
    stats = {
        "total_labels": len(label_files),
        "empty_files": 0,
        "no_image_match": 0,
        "files_with_issues": 0,
        "issue_lines_total": 0,
        "reasons_count": Counter(),
    }
    samples = []  # (file, line_no, reasons, line_text)

    for lf in label_files:
        stem = os.path.splitext(lf)[0]
        if not has_image(stem):
            stats["no_image_match"] += 1

        with open(lf, "r", encoding="utf-8", errors="ignore") as f:
            lines = f.readlines()

        nonempty = [ln for ln in lines if ln.strip()]
        if len(nonempty) == 0:
            stats["empty_files"] += 1
            continue

        bad_in_file = False
        for i, raw in enumerate(lines, start=1):
            s = raw.strip()
            if not s:
                continue
            ok, reasons = check_line(raw)
            # 형식오류거나 경고사유 존재 시 기록
            if (not ok) or any(r in reasons for r in
                               ("has_tab","has_comma","has_alpha",
                                "negative_value","non_positive_wh","value_gt_1")):
                bad_in_file = True
                stats["issue_lines_total"] += 1
                for r in reasons:
                    stats["reasons_count"][r] += 1
                if len(samples) < max_samples:
                    samples.append((lf, i, ",".join(reasons) if reasons else "-", raw.rstrip()))

        if bad_in_file:
            stats["files_with_issues"] += 1

    # 콘솔 요약 출력
    print(f"\n=== {folder} ===")
    print(f"총 라벨 파일: {stats['total_labels']}")
    print(f"비어있는 라벨 파일: {stats['empty_files']}")
    print(f"이미지 없는 라벨 파일: {stats['no_image_match']}")
    print(f"문제 라인이 존재하는 라벨 파일 수: {stats['files_with_issues']}")
    print(f"문제 라인 총 개수: {stats['issue_lines_total']}")
    if stats["reasons_count"]:
        print("문제 유형 카운트:")
        for k,v in stats["reasons_count"].most_common():
            print(f"  - {k}: {v}")
    if samples:
        print("\n문제 샘플(최대 {}개):".format(len(samples)))
        for fpath, ln, why, txt in samples:
            print(f"  {fpath}:{ln} [{why}] -> {txt}")

    # 파일로 상세 리포트 저장
    report_path = os.path.join(folder, "label_check_report.txt")
    with open(report_path, "w", encoding="utf-8") as r:
        r.write(f"# Report for {folder}\n")
        r.write(f"total_labels: {stats['total_labels']}\n")
        r.write(f"empty_files: {stats['empty_files']}\n")
        r.write(f"no_image_match: {stats['no_image_match']}\n")
        r.write(f"files_with_issues: {stats['files_with_issues']}\n")
        r.write(f"issue_lines_total: {stats['issue_lines_total']}\n")
        r.write("reasons_count:\n")
        for k,v in stats["reasons_count"].most_common():
            r.write(f"  - {k}: {v}\n")
        r.write("\n# samples\n")
        for fpath, ln, why, txt in samples:
            r.write(f"{fpath}:{ln} [{why}] -> {txt}\n")
    print(f"리포트 저장: {report_path}")

for d in FOLDERS:
    check_folder(d)



=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721 ===
총 라벨 파일: 3511
비어있는 라벨 파일: 717
이미지 없는 라벨 파일: 0
문제 라인이 존재하는 라벨 파일 수: 0
문제 라인 총 개수: 0
리포트 저장: /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721/label_check_report.txt

=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250725 ===
총 라벨 파일: 1688
비어있는 라벨 파일: 340
이미지 없는 라벨 파일: 0
문제 라인이 존재하는 라벨 파일 수: 0
문제 라인 총 개수: 0
리포트 저장: /home/dw/ws_job_msislab/Golf_Project/data/Data_20250725/label_check_report.txt


In [26]:
import os

# 데이터 루트 경로만 지정하면 됨
DATA_DIRS = [
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250721",
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725",
]

for data_dir in DATA_DIRS:
    removed = 0
    total = 0
    for root, _, files in os.walk(data_dir):
        for f in files:
            if f.endswith(".txt"):
                total += 1
                path = os.path.join(root, f)
                if os.path.getsize(path) == 0:  # 내용이 전혀 없으면
                    os.remove(path)
                    removed += 1
    print(f"=== {data_dir} ===")
    print(f"총 txt: {total}, 삭제된 빈 txt: {removed}")



=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721 ===
총 txt: 3513, 삭제된 빈 txt: 0
=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250725 ===
총 txt: 1689, 삭제된 빈 txt: 340


In [27]:
import os, glob

DATA_DIRS = [
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250721",
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725",
]

def has_valid_yolo_line(path):
    try:
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            for raw in f:
                s = raw.strip()
                if not s:
                    continue
                parts = s.replace("\t"," ").replace(",", " ").split()
                if len(parts) != 5:
                    continue
                # 간단 숫자 검증
                try:
                    int(parts[0])
                    float(parts[1]); float(parts[2]); float(parts[3]); float(parts[4])
                    return True   # 유효한 YOLO 라인 발견
                except:
                    continue
    except Exception:
        # 읽기 오류 파일은 유효하지 않다고 간주
        return False
    return False

for data_dir in DATA_DIRS:
    total = 0
    removed = 0
    samples = []
    for root, _, files in os.walk(data_dir):
        for f in files:
            if not f.endswith(".txt"):
                continue
            if f == "classes.txt":
                continue
            total += 1
            p = os.path.join(root, f)
            if not has_valid_yolo_line(p):
                os.remove(p)
                removed += 1
                if len(samples) < 15:
                    samples.append(os.path.relpath(p, data_dir))
    print(f"=== {data_dir} ===")
    print(f"총 txt: {total}, 삭제된 무효 txt: {removed}")
    if samples:
        print("삭제 예시:", samples)


=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721 ===
총 txt: 3512, 삭제된 무효 txt: 718
삭제 예시: ['20250721_053640_0029_000013_f000120_1920x1536_2308.txt', '20250721_053640_0007_000151_f001500_1920x1536_871.txt', '20250721_053640_0030_000030_f000290_1920x1536_2825.txt', '20250721_053640_0023_000096_f000950_1920x1536_2154.txt', '20250721_053640_0032_000119_f001180_1920x1536_4052.txt', '20250721_053640_0036_000170_f001690_1920x1536_5595.txt', '20250721_053640_0034_000117_f001160_1920x1536_4510.txt', '20250721_053640_0032_000045_f000440_1920x1536_3835.txt', '20250721_053640_0032_000033_f000320_1920x1536_3802.txt', '20250721_053640_0032_000006_f000050_1920x1536_3725.txt', '20250721_053640_0030_000137_f001360_1920x1536_3126.txt', '20250721_053640_0035_000006_f000050_1920x1536_4693.txt', '20250721_053640_0035_000021_f000200_1920x1536_4732.txt', '20250721_053640_0023_000004_f000030_1920x1536_1912.txt', '20250721_053640_0022_000153_f001520_1920x1536_1824.txt']
=== /home/dw/ws_job_msislab/Go

In [29]:
import os

def count_images_and_labels(base_dir):
    # 집계할 이미지 확장자
    image_exts = {".jpg", ".jpeg", ".png"}
    for folder in sorted(os.listdir(base_dir)):
        folder_path = os.path.join(base_dir, folder)
        if not os.path.isdir(folder_path):
            continue

        img_count = 0
        txt_count = 0

        for fname in os.listdir(folder_path):
            fpath = os.path.join(folder_path, fname)
            if not os.path.isfile(fpath):
                continue

            ext = os.path.splitext(fname)[1].lower()
            if ext in image_exts:
                img_count += 1
            elif ext == ".txt" and fname != "classes.txt":
                txt_count += 1

        print(f"=== {folder_path} ===")
        print(f"이미지 개수: {img_count}")
        print(f"TXT 개수  : {txt_count}\n")

# 실행 예시
count_images_and_labels("/home/dw/ws_job_msislab/Golf_Project/data")


=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250721 ===
이미지 개수: 3511
TXT 개수  : 2794

=== /home/dw/ws_job_msislab/Golf_Project/data/Data_20250725 ===
이미지 개수: 1688
TXT 개수  : 1348



In [30]:
import os, glob, random, shutil, yaml
from collections import Counter, defaultdict

# ===== 설정 =====
SOURCES = [
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250721",
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725",
]
OUT_ROOT = "/home/dw/ws_job_msislab/Golf_Project/data/Data_merged_yolov8"  # 출력 루트
SPLITS = {"train": 0.8, "val": 0.1, "test": 0.1}  # 합 = 1.0
IMG_EXTS = (".jpg",".jpeg",".png",".bmp",".webp")
NC = 3
NAMES = ["class0","class1","class2"]  # 필요시 ["divot", "class1", "class2"] 등으로 수정

random.seed(42)

# ===== 유틸 =====
def stem_of(p): return os.path.splitext(os.path.basename(p))[0]
def img_exists_for(stem, root):
    for e in IMG_EXTS:
        p = os.path.join(root, stem + e)
        if os.path.exists(p): return p
    return None

def read_yolo_labels(txt_path):
    """ YOLO 라벨 라인들 반환(유효한 5토큰만) """
    lines = []
    with open(txt_path, "r", encoding="utf-8", errors="ignore") as f:
        for raw in f:
            s = raw.strip()
            if not s: 
                continue
            parts = s.split()
            if len(parts) != 5:
                continue
            try:
                _cid = int(parts[0])
                float(parts[1]); float(parts[2]); float(parts[3]); float(parts[4])
            except:
                continue
            lines.append(parts)
    return lines  # [["0","xc","yc","w","h"], ...]

def primary_class_from(lines):
    """ 한 이미지의 대표 클래스(가장 많이 등장한 class id) """
    if not lines: 
        return None
    cnt = Counter(int(x[0]) for x in lines)
    return cnt.most_common(1)[0][0]

def ensure_dirs():
    for split in ("train","val","test"):
        os.makedirs(os.path.join(OUT_ROOT, "images", split), exist_ok=True)
        os.makedirs(os.path.join(OUT_ROOT, "labels", split), exist_ok=True)

def copy_pair(img_path, lbl_path, split):
    dst_img = os.path.join(OUT_ROOT, "images", split, os.path.basename(img_path))
    shutil.copy2(img_path, dst_img)
    if lbl_path and os.path.exists(lbl_path):
        dst_lbl = os.path.join(OUT_ROOT, "labels", split, os.path.basename(lbl_path))
        shutil.copy2(lbl_path, dst_lbl)

# ===== 1) 소스 통합 목록 만들기 =====
samples = []  # dict: {"img":path, "lbl":path_or_None, "primary":int|None}
for src in SOURCES:
    # 라벨 기준으로 먼저 추가
    lbls = [p for p in glob.glob(os.path.join(src, "*.txt")) if os.path.basename(p) != "classes.txt"]
    seen = set()
    for lp in lbls:
        st = stem_of(lp)
        ip = img_exists_for(st, src)
        if not ip: 
            continue  # 방어적
        lines = read_yolo_labels(lp)
        primary = primary_class_from(lines)
        samples.append({"img": ip, "lbl": lp if lines else None, "primary": primary})
        seen.add(st)
    # 라벨이 없는 이미지(배경) 추가
    for e in IMG_EXTS:
        for ip in glob.glob(os.path.join(src, f"*{e}")):
            st = stem_of(ip)
            if st in seen: 
                continue
            # 라벨 파일이 없으면 배경으로
            lp = os.path.join(src, st + ".txt")
            if not os.path.exists(lp):
                samples.append({"img": ip, "lbl": None, "primary": None})

total_imgs = len(samples)
bg_count = sum(1 for s in samples if s["lbl"] is None)
print(f"[수집] 총 이미지 {total_imgs}장, 배경(라벨없음) {bg_count}장 ({bg_count/total_imgs*100:.2f}%)")

# ===== 2) split을 위한 버킷 구성 (라벨 유/무 분리, 라벨 있는 쪽은 클래스별 분할) =====
labeled_by_class = defaultdict(list)
background = []
for s in samples:
    if s["lbl"] is None:
        background.append(s)
    else:
        # primary가 None일 일은 거의 없지만 방어
        labeled_by_class[s["primary"] if s["primary"] is not None else -1].append(s)

# ===== 3) 분할 함수 =====
def split_list(lst, ratios):
    n = len(lst)
    n_train = int(n * ratios["train"])
    n_val = int(n * ratios["val"])
    # 남는 건 test
    idx = list(range(n))
    random.shuffle(idx)
    tr_idx = idx[:n_train]
    va_idx = idx[n_train:n_train+n_val]
    te_idx = idx[n_train+n_val:]
    return ([lst[i] for i in tr_idx],
            [lst[i] for i in va_idx],
            [lst[i] for i in te_idx])

# ===== 4) 실제 분할 (라벨 있는 쪽: 클래스별로 동일 비율 유지 / 배경도 동일 비율) =====
splits = {"train": [], "val": [], "test": []}

# 라벨 있는 샘플: 클래스별로 나눠서 합치기(라벨 분포 유지)
for cls_id, lst in labeled_by_class.items():
    tr, va, te = split_list(lst, SPLITS)
    splits["train"].extend(tr)
    splits["val"].extend(va)
    splits["test"].extend(te)

# 배경 샘플도 동일 비율로 배분
tr, va, te = split_list(background, SPLITS)
splits["train"].extend(tr)
splits["val"].extend(va)
splits["test"].extend(te)

# ===== 5) 저장(복사) =====
ensure_dirs()
for split, lst in splits.items():
    for s in lst:
        copy_pair(s["img"], s["lbl"], split)

# ===== 6) data.yaml 생성 =====
data_yaml = {
    "path": OUT_ROOT,
    "train": "images/train",
    "val": "images/val",
    "test": "images/test",
    "nc": NC,
    "names": NAMES,
}
os.makedirs(OUT_ROOT, exist_ok=True)
with open(os.path.join(OUT_ROOT, "data.yaml"), "w", encoding="utf-8") as f:
    yaml.safe_dump(data_yaml, f, allow_unicode=True, sort_keys=False)

# ===== 7) 요약 리포트 =====
def count_split(split):
    img_dir = os.path.join(OUT_ROOT, "images", split)
    lbl_dir = os.path.join(OUT_ROOT, "labels", split)
    imgs = [p for p in glob.glob(os.path.join(img_dir, "*")) if os.path.splitext(p)[1].lower() in IMG_EXTS]
    lbls = [p for p in glob.glob(os.path.join(lbl_dir, "*.txt"))]
    # 배경 = 이미지 중 대응 라벨이 없는 것
    has_lbl = set(stem_of(p) for p in lbls)
    bg = [p for p in imgs if stem_of(p) not in has_lbl]
    return len(imgs), len(lbls), len(bg)

print("\n=== 분할 결과 ===")
for sp in ("train","val","test"):
    ni, nl, nb = count_split(sp)
    print(f"{sp}: images={ni}, labels={nl}, background(images-no-txt)={nb}")

print(f"\n[완료] 병합 데이터셋이 준비되었습니다 → {OUT_ROOT}")
print(f"- data.yaml: {os.path.join(OUT_ROOT, 'data.yaml')}")


[수집] 총 이미지 5199장, 배경(라벨없음) 1057장 (20.33%)

=== 분할 결과 ===
train: images=4157, labels=3312, background(images-no-txt)=845
val: images=517, labels=412, background(images-no-txt)=105
test: images=525, labels=418, background(images-no-txt)=107

[완료] 병합 데이터셋이 준비되었습니다 → /home/dw/ws_job_msislab/Golf_Project/data/Data_merged_yolov8
- data.yaml: /home/dw/ws_job_msislab/Golf_Project/data/Data_merged_yolov8/data.yaml
